In [1]:
# UNet网络结构，参考资料：https://cuijiahua.com/blog/2019/12/dl-15.html
# 上采样和下采样https://blog.csdn.net/Miss_yuki/article/details/53286735

In [2]:
import torch.nn as nn

# UNet中有许多两次卷积操作，所以单独写一个DoubleConv模块
class DoubleConv(nn.Module):
    # (convolution => [BN] => ReLU)*2
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=0),#3*3卷积，https://blog.csdn.net/qq_34107425/article/details/104117670
            nn.BatchNorm2d(out_channels),# 数据归一化，https://blog.csdn.net/bigFatCat_Tom/article/details/91619977
            nn.ReLU(inplace=True),# 激活函数 https://www.jianshu.com/p/2d5e1331500b
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)

In [3]:
class Down(nn.Module):
    # 下采样downscaling with maxpool then double conv
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),#池化，https://www.jianshu.com/p/9d93a3391159
            DoubleConv(in_channels, out_channels)
        )
    
    def forward(self, x):
        return self.maxpool_conv(x)

In [4]:
import torch.nn.functional as F

class Up(nn.Module):
    #上采样Upscaling then double conv
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        # if bilinear(双线性), use the normal convolutions to reduce the number of channels
        if bilinear:
            # 双线性插值 https://zhuanlan.zhihu.com/p/49832048
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            # 反卷积
            self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2,stride=2)
        
        self.conv = DoubleConv(in_channels, out_channels)
    
    def forward(self, x1, x2):
        x1 = self.up(x1)
        # 特征的融合
        diffY = torch.tensor([x2.size()[2]-x1.size()[2]])
        diffX = torch.tensor([x2.size()[3]-x1.size()[3]])
        
        # 填充
        x1 = F.pad(x1, [diffX//2, diffX-diffX//2,
                        diffY//2, diffY-diffY//2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [5]:
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
    def forward(self, x):
        return self.conv(x)

In [6]:
# UNet模型
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.up1 = Up(1024, 512, bilinear)
        self.up2 = Up(512, 256, bilinear)
        self.up3 = Up(256, 128, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)
    
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [7]:
# 测试
net = UNet(n_channels=3, n_classes=1)
print(net)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): Re

In [ ]:
# OVER